In [1]:
# Use modules from parent directory
import os
import sys
import tifffile as tiff
import numpy as np
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000   
from matplotlib import pyplot as plt

sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2
from srcv2_2.models.params import get_params
from srcv2_2.models.Unet import Unet, UnetV2
from srcv2_2.visualization.make_image_files import visualize_test_data, __visualize_landsat8_tile__
from srcv2_2.visualization.visualization_utils import get_predicted_thumbnails 
from srcv2_2.utils import get_model_name

import tensorflow as tf

2024-02-23 14:29:08.932735: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-23 14:29:08.950369: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 14:29:08.950384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 14:29:08.950791: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-23 14:29:08.953903: I tensorflow/core/platform/cpu_feature_guar

## Load KERAS Model

In [2]:
params = get_params('U-net', 'Landsat8')
params.cls = ['cloud', 'thin'] # clear, cloud, shadow, snow, water
params.project_path = "/home/mxh/RS-Net/"
activation_func='relu'
params.learning_rate =1e-3
params.epochs = 5
params.threshold = 0.5
params.dropout = 0
params.L2reg = 0
params.decay = 0
params.num_gpus = 1
params.modelID = "Test_Local"
params.norm_threshold = 3500

#tile = 'all'  # Name of one tile or 'all'
#tile = 'L2A_T32UNF_20170803T103019' 

In [3]:
local_loaded_model = tf.keras.saving.load_model(f"../models/Unet/{get_model_name(params)}.keras")
# weights and funcs will automatically be loaded

unet_loaded_local_model = UnetV2(params, model=local_loaded_model)

2024-02-23 14:29:12.844409: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-23 14:29:12.902516: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-23 14:29:12.902601: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

## Load MODIS file(s)

In [4]:
modis_path = "/media/mxh/Volume/SAT-DATA/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/"
day1 = "MOD09GA.A2023335.h18v04.061.2023340033530.hdf"

from osgeo import gdal

In [5]:
day1_hdf = gdal.Open(modis_path + day1)
day1_subsets = day1_hdf.GetSubDatasets()

/home/mxh/anaconda3/envs/tf2+gpu/lib/python3.11/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [6]:
day1_subsets

[('HDF4_EOS:EOS_GRID:"/media/mxh/Volume/SAT-DATA/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/MOD09GA.A2023335.h18v04.061.2023340033530.hdf":MODIS_Grid_1km_2D:num_observations_1km',
  '[1200x1200] num_observations_1km MODIS_Grid_1km_2D (8-bit integer)'),
 ('HDF4_EOS:EOS_GRID:"/media/mxh/Volume/SAT-DATA/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/MOD09GA.A2023335.h18v04.061.2023340033530.hdf":MODIS_Grid_1km_2D:state_1km_1',
  '[1200x1200] state_1km_1 MODIS_Grid_1km_2D (16-bit unsigned integer)'),
 ('HDF4_EOS:EOS_GRID:"/media/mxh/Volume/SAT-DATA/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/MOD09GA.A2023335.h18v04.061.2023340033530.hdf":MODIS_Grid_1km_2D:SensorZenith_1',
  '[1200x1200] SensorZenith_1 MODIS_Grid_1km_2D (16-bit integer)'),
 ('HDF4_EOS:EOS_GRID:"/media/mxh/Volume/SAT-DATA/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/MOD09GA.A2023335.h18v04.061.2023340033530.hdf":MODIS_Grid_1km_2D:SensorAzimuth_1',
  '[1200x1200] SensorAzimuth_1 MODIS_Grid_1km_2D (16-bit integer)'),
 ('HDF4_EOS:EOS_GRID:"/media/mxh/Vol

In [7]:
day1_state_1km = gdal.Open('HDF4_EOS:EOS_GRID:"/media/mxh/Volume/SAT-DATA/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/MOD09GA.A2023335.h18v04.061.2023340033530.hdf":MODIS_Grid_1km_2D:state_1km_1')
day1_sur_refl_b06 = gdal.Open('HDF4_EOS:EOS_GRID:"/media/mxh/Volume/SAT-DATA/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/MOD09GA.A2023335.h18v04.061.2023340033530.hdf":MODIS_Grid_500m_2D:sur_refl_b06_1')

In [ ]:
# convert to TIFF for Unet input?

## Run KERAS Model on MODIS file